In [1]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, Column, Integer, String

engine = create_engine('sqlite:///:memory:')
Base = declarative_base()

In [2]:
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
            self.name, self.fullname, self.nickname)

In [3]:
Base.metadata.create_all(engine)

## use contextmanager

In [4]:
from contextlib import contextmanager
@contextmanager
def session_scope(engine):
    """Provide a transactional scope around a series of operations."""
    sess_cls = sessionmaker(bind=engine)
    session = sess_cls()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [5]:
# table is empty
with session_scope(engine) as session:
    print(session.query(User).all())

[]


In [6]:
with session_scope(engine) as session:
    # add one item, will be commited if no error raised
    ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')
    session.add(ed_user)

In [7]:
# check commit
print(session.query(User).all())

[<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>]


In [8]:
try:
    with session_scope(engine) as session:
        # modify item & not commit
        ed_user = session.query(User).first()
        ed_user.name = 'Edwardo'
        print(session.query(User).all())
        
        # encounter an error, will rollback all uncommited changes
        _ = 0/0
except Exception as e:
    print(e)

[<User(name='Edwardo', fullname='Ed Jones', nickname='edsnickname')>]
integer division or modulo by zero


In [9]:
# check rollback
with session_scope(engine) as session:
    print(session.query(User).all())

[<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>]
